In [7]:
import pandas as pd
import codecs
import os,subprocess
import re
from collections import deque

In [125]:
#df_usr = pd.read_csv('usr_Geo_ncert_6stnd_4ch_0005.txt',sep=',',header=None)

In [23]:
pwd()

'/Users/adityaparashar/LTRC'

In [122]:
#df_usr.to_excel('usr_Geo_ncert_6stnd_4ch_0005.xlsx')

In [8]:
def getLines(file,splitby):
    with codecs.open(file, 'r', encoding="utf-8", errors='ignore') as filer:
                    data = filer.read()
                    data = re.sub('\t+', '\t', data)
    data = data.strip('\n')
    lines = data.split(splitby)
    return lines

In [9]:
class NewNode():
 
    def __init__(self, val,label, r3, gnp, rel, disc, r7, r8):
        self.key = val
        self.child =[]
        self.label = label
        self.r3 = r3
        self.gnp = gnp
        self.rel = rel
        self.disc = disc
        self.r7 = r7
        self.r8 = r8
        

In [136]:
node_dict = {}
def get_tree(id,lines):
    try:
        labels = lines[0].split(',')
        concs = lines[1].split(',')
        deps = lines[4].split(',')
        rels = [i.split(':')[1] for i in deps]
        r3s = lines[2].split(',')
        gnps = lines[3].split(',')
        disc_els = lines[5].split(',')
        r7s = lines[6].split(',')
        r8s = lines[7].split(',')
        conj = []
        if len(lines) == 10:
            if lines[9][:4] == 'conj':
                conj = re.findall(r'\[(.*?)\]', lines[9])[0].split(',')
        nodes = []
        for i in range(len(concs)):
            nodes.append(NewNode(concs[i],labels[i],r3s[i],gnps[i],rels[i],disc_els[i],r7s[i],r8s[i]))

        parents = [i.split(':')[0] for i in deps]
        for i in range(len(parents)):
            if parents[i] == '0':
                rootnode = nodes[i]
            else:
                if len(conj)>0 and i+1 == conj[0]:
                    for j in range(len(conj)):
                        nodes[int(parents[conj[j]-1])-1].child.append(nodes[conj[j]-1])
                elif len(conj)>0 and i+1 in conj:
                    continue
                else:
                    nodes[int(parents[i])-1].child.append(nodes[i])
        return rootnode
    except:
        print(f'{id}')
        return 0
#get_tree(1,usr_lines).key
#rootnode.child

In [120]:
def get_speaker_vp(node,usr_id):
    try:
        sp_vp = node.r7
        vps = ['def','respect','informal','emph']
        if sp_vp in vps:
            return f'\n:attr_{sp_vp}(yes)'
        elif sp_vp!='' and ':' in sp_vp:
            print([sp_vp,node.label,usr_id])
            disc = sp_vp.split(':')[1]
            return f'\n:disc_part({disc})'
        return ''
    except:
        print(f'failed get_speaker_vp for  {usr_id}')

In [122]:
def get_conj(usr_lines):
    try:
        conj = []
        if len(usr_lines) == 10:
                if usr_lines[9][:4] == 'conj':
                    conj = re.findall(r'\[(.*?)\]', usr_lines[9])[0].split(',')
        return conj
    except:
        print(f'failed conjunction for  {usr_lines}')

In [123]:
def get_senttype(usr_lines):
    try:
        if len(usr_lines) ==10:
            return usr_lines[-2]
        elif len(usr_lines) == 9:
            return usr_lines[-1]
    except:
        print(f'failed sent type for  {usr_lines}')

In [124]:
def get_gnp_value(node):
    if node.gnp != '':
        return re.findall(r'\[(.*?)\]', node.gnp)[0]
    else:
        return ''

In [125]:
comparison_markers = ['superl','comper-less','comper-more']
caus_markers = ['causative','double_causative']
def get_gnp(node):
    try:
        if node.gnp != '':
            gnp = re.findall(r'\[(.*?)\]', node.gnp)[0]
            if ' ' in gnp:
                gen = gnp.split(' ')[0]
                num = gnp.split(' ')[1]
                per = gnp.split(' ')[2]
                return f'\n:attr_gen({gen})\n:attr_num({num})\n:attr_per({per})'
            elif gnp in comparison_markers:
                return f'\n:attr_comparison({gnp})'
            #elif gnp in caus_markers:
                #return f':attr_{gnp}(yes)'
            else:
                return ''
                print(f'new gnp found {node.gnp}')
        else:
            return ''
    except:
        print(f'failed gnp for  {node.gnp}')

In [126]:
def get_anim(node):
    if node.r3 == 'anim':
        return '\n:attr_anim(yes)'
    elif node.r3 != '':
        if node.r3 in ['mass','abs']:
            return f'\n:attr_sem-cat({node.r3})'
        else:
            return f'\n:attr_ne({node.r3})'
    else:
        return ''

In [127]:
def get_tam(node, usr_id):
    tam = ''
    if '-' in node.label:
        tam = node.label.split('-')[1]
        return f'\n:TAM({tam})'
    else:
        #print(f'no tam {node.label} {usr_id}')
        return tam

In [61]:
interr_concepts = ['kyA' , 'kOna', 'kahAz', 'kEse', 'kyoM', 'kaba', 'kOna', 'kOnasA', 'kise', 'kisase', 'kisako', 'kisakA','kisakI', 'kisalie', 'kiwanA']
def get_label(usr_id,node,usr_dict):
    try:
        if ':coref' in node.disc:
            ref_usr = node.disc.split(':')[0]
            if '.' in ref_usr:
                ref_usr_id = ref_usr.split('.')[0]
                ref_concid = ref_usr.split('.')[1]
                if ref_usr_id in usr_dict.keys():
                    if usr_dict[ref_usr_id][4].split(',')[int(ref_concid)-1].split(':')[1] == 'main':
                        return '\n:attr_coref(yes)' + preorderTraversal(ref_usr_id,get_tree(ref_usr_id,usr_dict[ref_usr_id]),get_senttype(usr_dict[ref_usr_id]),usr_dict)
                    else:
                        return usr_dict[ref_usr_id][0].split(',')[int(ref_concid)-1] +'\n'+':attr_coref(yes)'
                else:
                    print(f'coreffed USR {ref_usr_id} not present in the data for {usr_id}')
                    return '\n:attr_coref(yes)' + node.label
                    
            else:
                ref_concid = ref_usr
                return usr_dict[usr_id][0].split(',')[int(ref_concid)-1] +'\n'+':attr_coref(yes)'
        elif get_senttype(usr_dict[usr_id]) == 'interrogative' and node.label.split('_')[0] in interr_concepts:
            return 'unknown'
        elif ':' in node.disc:
            print([usr_id,node.disc])
            return node.label
        else:
            return node.label
    except:
        print(f'failed label for {usr_id}')

In [112]:
def build_linear_node(usr_id,node,usr_dict):
    if node.rel == 'main':
        rel = ''
        
        if get_gnp_value(node) in caus_markers:
            rel = get_gnp_value(node)
        return rel + '(' + node.label.split('-')[0]  + get_tam(node,usr_id) +get_anim(node)  + get_gnp(node) +get_speaker_vp(node,usr_id) 
        
    else:
        conj = get_conj(usr_dict[usr_id])
        if node.key in conj:
            if node.key == conj[0]:
                return '\n:'+node.rel + '(' 'op1'+'(' + get_label(usr_id,node,usr_dict) + get_anim(node) + get_gnp(node) + get_speaker_vp(node,usr_id)
            else:
                opindex = conj.index(node.key) +1
                return '\n:'+ f'op{opindex}'+'(' + get_label(usr_id,node,usr_dict)  +get_anim(node) + get_gnp(node) +get_speaker_vp(node,usr_id)
        
        if get_gnp_value(node) in caus_markers:
            print(usr_id,node.label)
        return '\n:'+ node.rel + '(' + get_label(usr_id,node,usr_dict)  +get_anim(node) + get_gnp(node)  +get_speaker_vp(node,usr_id) 

In [138]:
def preorderTraversal(usr_id,root,sent_type,usr_dict):
    try:
        stack = deque([])
        lin = ''
        preorder =[]
        preorder.append(root.key)
        conj = get_conj(usr_dict[usr_id])
        lin+= build_linear_node(usr_id,root,usr_dict)
        stack.append(root)
        while len(stack)>0:
            flag = 0
            if len((stack[len(stack)-1]).child)== 0:
                X = stack.pop()
                if len(conj)>0:
                    #print(conj)
                    if X.key == conj[-1]:
                        lin += ')'

                lin+= ')'
            else:
                par = stack[len(stack)-1]
            for i in range(0, len(par.child)):
                if par.child[i].key not in preorder:
                    flag = 1
                    stack.append(par.child[i])
                    preorder.append(par.child[i].key)
                    lin += build_linear_node(usr_id,par.child[i],usr_dict)
                    break;
            if flag == 0:
                node = stack.pop()
                if node.rel == 'main':
                    lin += f'\n:CLAUSE_TYPE({sent_type}))'
                else:
                    if len(conj)>0:
                        if node.key == conj[-1]:
                            lin += ')'
                    lin+= ')'
        return lin
    except:
        print(f'{usr_id}')
    #print(preorder)
    #print(lin)

In [ ]:
for path in os.listdir('bulk_USRs_mod'):
    if '_9stnd' in path:
        npath = path.replace('_9stnd','_09stnd')
        os.rename(my_source, my_dest)
        

In [259]:
%cd ..
#pwd()

/Users/adityaparashar/LTRC


In [36]:
def build_usr_dict(dir_path,isFileNameint):
    usr_dict = {}
    if isFileNameint:
        temp = [int(i) for i in os.listdir(dir_path) if '.' not in i]
        sort = sorted(temp)
        file_list = [str(i) for i in sort]
    
    else:
        file_list = sorted(os.listdir(dir_path))
    for path in file_list:
        if os.path.isfile(os.path.join(dir_path, path)):
                fpath = dir_path.split('/')[-1] + '/' + path
                fname = path.split('.')[0]
                if '.' in path:
                    print(path)
                if fname in usr_dict.keys():
                    print(f'USR {fname} already in the dictionary')
                else:
                    usr_dict[fname] = getLines(fpath,'\n')
    return usr_dict
usr_dict = build_usr_dict('bulk_USRs_mod',False)
usr_dict2 = {k: usr_dict[k][1:] for k in usr_dict.keys()}
#print(usr_dict.keys())
#print([usr_dict['1'],usr_dict2['1']])
#build_usr_dict('verified_sent')

In [37]:
len(usr_dict)

3455

In [39]:
from collections import OrderedDict
usr_dict = {}
def build_lin_usr_dict(usr_folder_name,isFileNameint):
    lin_usr_dict = {}
    failed = []
    usr_dict0 = build_usr_dict(usr_folder_name,isFileNameint)
    usr_dict = {k: usr_dict0[k][1:] for k in usr_dict0.keys()}
    #print(usr_dict.keys())
    for key in usr_dict.keys():
        tree = get_tree(key,usr_dict[key])
        if tree != 0:
            sent_type = get_senttype(usr_dict[key])
            if preorderTraversal(key,tree,sent_type,usr_dict) is not None:
                lin_usr_dict[key]= os.linesep.join([s for s in preorderTraversal(key,tree,sent_type,usr_dict).splitlines() if s])
            else:
                failed.append(key)
        else:
            failed.append(key)
            continue
    return lin_usr_dict,usr_dict0,failed
[lin_usr_dict,usr_dict,failed] = build_lin_usr_dict('verified_sent',True)

['39', '2:deic']
['39', '2:deic']
['120', '2:deic']
['120', '2:deic']
150
['180', '2:deic']
['180', '2:deic']
307
310
367
368
369
370


In [33]:
lin_usr_dict

{'1': '(raha_1\n:TAM(wA_hE_1)\n:k1(addressee\n:attr_anim(yes)\n:attr_gen(m)\n:attr_num(sg)\n:attr_per(m)\n:attr_respect(yes))\n:k7p(unknown)\n:CLAUSE_TYPE(interrogative))',
 '2': '(A_1\n:TAM(yA_hE_1)\n:k1(aXyApaka_1\n:attr_anim(yes)\n:attr_gen(m)\n:attr_num(sg)\n:attr_per(a)\n:attr_def(yes))\n:kr_vn(aBI_1)\n:CLAUSE_TYPE(affirmative))',
 '3': '(KA_1\n:TAM(wA_hE_1)\n:k1(baccA_3\n:attr_anim(yes)\n:attr_gen(-)\n:attr_num(sg)\n:attr_per(a))\n:k7t(agaswa)\n:k2(Pala_1\n:attr_gen(-)\n:attr_num(pl)\n:attr_per(a))\n:CLAUSE_TYPE(affirmative))',
 '4': '(KA_1\n:TAM(wA_hE_1)\n:k1(baccA_3\n:attr_anim(yes)\n:attr_gen(-)\n:attr_num(pl)\n:attr_per(a))\n:k7t(janavarI)\n:k2(Pala_1\n:attr_gen(-)\n:attr_num(pl)\n:attr_per(a))\n:CLAUSE_TYPE(affirmative))',
 '5': '(KA_1\n:TAM(wA_hE_1)\n:k1(baccA_3\n:attr_anim(yes)\n:attr_gen(-)\n:attr_num(sg)\n:attr_per(a))\n:k2(unknown)\n:CLAUSE_TYPE(interrogative))',
 '6': '(KA_1\n:TAM(wA_hE_1)\n:k1(baccA_3\n:attr_anim(yes)\n:attr_gen(-)\n:attr_num(pl)\n:attr_per(a))\n:k7t(